# Exploring conformational space of selected macrocycles - "M1", Part 2

In this notebook we present and analyze selected structures, technical notes are [here](www.gitlab.com/user/gosia/icho).

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [29]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Mon Apr  3 13:39:22 2017


In [3]:
# Functions used in this notebook:
def grep_energies_from_csvfile(fcsv):
    energies = []
    names = []
    with open(fcsv,'r') as f:
        lines = f.readlines()
        for line in lines:
            words = line.split()
            energies.append(float(words[1]))
            names.append(words[0])
    return names, energies

In [4]:
# decide what is the "core" - a part of molecule, which we wish to be most aligned (rmsd-wise) among all the structures
# here: pyridine ring
m1 = Chem.AddHs(Chem.MolFromSmiles('O=C1NCCNC(=O)c2nc(C(=O)NCCNC(=O)c3nc1ccc3)ccc2'))
core_m1 = m1.GetSubstructMatch(Chem.MolFromSmiles('n1ccccc1'))

templ_m1 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal.sdf')
m1_crystal = templ_m1[0]

### Pre-optimization:

Chosen conformers:

In [6]:
# write conformers as a list
allmol_m1_b_sdf = []
allmol_m1_b_smi = []
suppl_m1_b_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal_sdfout.sdf')
suppl_m1_b_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal_smilesout.sdf')

for mol in suppl_m1_b_sdf:
    allmol_m1_b_sdf.append(mol)
for mol in suppl_m1_b_smi:
    allmol_m1_b_smi.append(mol)    

In [7]:
selected_b = [allmol_m1_b_sdf[0], allmol_m1_b_sdf[2], allmol_m1_b_sdf[3], allmol_m1_b_smi[2], allmol_m1_b_smi[3], allmol_m1_b_smi[6]]

for mol in selected_b:
    #AllChem.AlignMolConformers(mol,atomIds=core_m1)
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles('n1ccccc1'))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p_b = py3Dmol.view(width=400,height=400)
for mol in selected_b:
    mb = Chem.MolToMolBlock(mol)
    p_b.addModel(mb,'sdf')
p_b.setStyle({'stick':{'radius':'0.15'}})
p_b.setBackgroundColor('0xeeeeee')
p_b.zoomTo()
p_b.show()

### Geometry optimization in Gaussian 09 software

**Computational setup:**

* Gaussian 09: DFT/PBE 6-31G(d)
* geometry optimization keywords: 
` #p PBEh1PBE/6-31G(d') opt test nosymm pop=full`
* frequency calculations keywords:
` #p PBEh1PBE/6-31G(d') freq test geom=check guess=read`

In [8]:
inps_m1_b_gaussian = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/gaussian_results/*.sdf')
allmol_m1_b_gaussian = []

for inp in inps_m1_b_gaussian:
    mol = Chem.MolFromMolFile(inp)
    allmol_m1_b_gaussian.append(mol)
print(len(allmol_m1_b_gaussian))    

for mol in allmol_m1_b_gaussian:
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles('n1ccccc1'))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p = py3Dmol.view(width=400,height=400)
for mol in allmol_m1_b_gaussian:
    mb = Chem.MolToMolBlock(mol)
    p.addModel(mb,'sdf')
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()    


6


From the alignment presented above we see that there are 4 distinct conformers, which we can identify on the list as:

In [27]:
f = "/home/gosia/work/work_on_gitlab/icho/calcs/m1/gaussian_results/energy.csv"
name, e_m1_b_gaussian = grep_energies_from_csvfile(f)
for i, mol in enumerate(allmol_m1_b_gaussian):
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(name[i].rsplit("_", 5)[0], e_m1_b_gaussian[i], AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m1_crystal))))

name = m1_b_smi_6, E = -1324.897293, RMS = 0.825545
name = m1_b_smi_2, E = -1324.913266, RMS = 1.257003
name = m1_b_sdf_3, E = -1324.913325, RMS = 1.258224
name = m1_b_sdf_2, E = -1324.915654, RMS = 1.216957
name = m1_b_sdf_0, E = -1324.913316, RMS = 1.256775
name = m1_b_smi_3, E = -1324.913265, RMS = 1.258173


From the similar "m1_b_smi_2", "m1_b_sdf_3" and "m1_b_sdf_0", we will select "m1_b_sdf_3" which has the lowest energy.